# Configs

In [1]:
MODEL_PATH = "./models/ppo/ppo"
TENSORBOARD_LOG_DIR = "./models/ppo/logs"
IMAGE_TAG = "ppo_rewards"
IMAGE_DIR = "./images/ppo"
RUNS_FILE = "./models/ppo/ppo_success_runs.pkl"

#### ENV CONFIGS ####
CONFIG = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 15,  # Number of other vehicles to observe
        "features": ["presence", "x", "y", "vx", "vy"],  # Observe position and velocity
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-10, 10],
            "vy": [-10, 10]
        },
        "absolute": False,
        "clip": False,
        "normalize": False
    },
    "action": {
        "type": "DiscreteMetaAction",  # Keep simple, 5 discrete actions
    },
    "simulation_frequency": 10,
    "policy_frequency": 10,
    "destination": "o1",
    "initial_vehicle_count": 20,
    "spawn_probability": 0.8,
    "ego_spacing": 25,
    "initial_lane_id": None,
    "controlled_vehicles": 1,
    "duration": 15,  # seconds
    "vehicles_density": 1.0,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.6],
    "scaling": 5.5 * 1.3,
    "normalize_reward": False
}

# Imports

In [2]:
import time
import torch
import pickle
import random
import gymnasium as gym

from fvcore.nn import FlopCountAnalysis

from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import ProgressBarCallback

from custom_intersection_env import CustomIntersectionEnv
from custom_training_callback import RewardTrackingCallback

/Users/preranagowda/Desktop/cse-190-final-project/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


# Register Env with Gymnasium

In [4]:
gym.envs.registration.register(
    id='custom-intersection-v0',
    entry_point='custom_intersection_env:CustomIntersectionEnv',
)

/Users/preranagowda/Desktop/cse-190-final-project/.venv/lib/python3.13/site-packages/gymnasium/envs/registration.py:644: UserWarning: WARN: Overriding environment custom-intersection-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


# Create + Wrap

In [5]:
env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=CONFIG)
env = DummyVecEnv([lambda: env])

# Set up Model

In [7]:
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=5,
    batch_size=5,
    learning_rate=7e-4,
    gamma=0.99,
    gae_lambda=1.0,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    normalize_advantage=False,
    tensorboard_log=TENSORBOARD_LOG_DIR,
    verbose=0,
    device="cpu",
)

# Training 

In [8]:
reward_callback = RewardTrackingCallback(
    tag=IMAGE_TAG,
    path_dir=IMAGE_DIR
)

destinations = ["o1", "o2", "o3"]
steps = [20000, 20000, 20000]

for dest, steps in zip(destinations, steps):
    config = CONFIG.copy()
    config["destination"] = dest  # Change destination for each training phase
    env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=config)
    env = DummyVecEnv([lambda: env])
    model.set_env(env)  # Update the model with the new environment
    model.learn(
        total_timesteps=steps,
        callback=[ProgressBarCallback(), reward_callback]
    )
    reward_callback.start_new_phase()
reward_callback.save_all_plot()

Output()

Output()

Output()

In [9]:
model.save(MODEL_PATH)

In [10]:
model = PPO.load(MODEL_PATH)

In [11]:
collisions = 0
destination_arrivals = 0
success_count = 0
successful_flopcount = 0
episodes = 100

# Store successful runs for rendering
successful_runs = []

for eps in range(100):
    config = CONFIG.copy()
    config["destination"] = "o" + str(random.randint(1, 3))
    env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=config)

    obs, _ = env.reset()
    episode_flops = 0
    done = False
    truncated = False
    episode_reward = 0
    trajectory = []

    while not (done or truncated):
        # Flop Counting
        input_tensor, _ = model.policy.obs_to_tensor(obs)
        flops = FlopCountAnalysis(model.policy, input_tensor)
        flops.unsupported_ops_warnings(False)
        flops = flops.total()
        episode_flops += flops

        action, _states = model.predict(obs, deterministic=True)
        trajectory.append((obs, action))  # Save for later render if successful
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward
    
    crashed = info.get("crashed", False)
    arrived = info.get("arrived", False)
    if crashed:
        collisions += 1
    if arrived:
        destination_arrivals += 1
    if (not crashed) and arrived:
        success_count += 1
        successful_flopcount += episode_flops
        successful_runs.append((config.copy(), trajectory))

    print(f"Episode {eps + 1} finished, total reward: {episode_reward}, destination: {config['destination']}")
    time.sleep(1)

print(f"Total collisions: {collisions} out of {episodes} episodes")
print(f"Total destination arrivals: {destination_arrivals} out of {episodes} episodes")
if success_count > 0:
    print(f"FLOPS per successful episode: {successful_flopcount / success_count:.2}")
else:
    print("No successful episodes.")



Episode 1 finished, total reward: -25.0, destination: o1
Episode 2 finished, total reward: -25.0, destination: o2
Episode 3 finished, total reward: 50.0, destination: o1
Episode 4 finished, total reward: -25.0, destination: o1
Episode 5 finished, total reward: 50.0, destination: o2
Episode 6 finished, total reward: -25.0, destination: o3
Episode 7 finished, total reward: -25.0, destination: o2
Episode 8 finished, total reward: -25.0, destination: o3
Episode 9 finished, total reward: -25.0, destination: o2
Episode 10 finished, total reward: 50.0, destination: o1
Episode 11 finished, total reward: 50.0, destination: o1
Episode 12 finished, total reward: -25.0, destination: o2
Episode 13 finished, total reward: -25.0, destination: o2
Episode 14 finished, total reward: -25.0, destination: o3
Episode 15 finished, total reward: -25.0, destination: o2
Episode 16 finished, total reward: -25.0, destination: o3
Episode 17 finished, total reward: -25.0, destination: o2
Episode 18 finished, total 

# Save and Load Successful Runs
# Then Render 

In [12]:
with open(RUNS_FILE, "wb") as f:
    pickle.dump(successful_runs, f)

In [13]:
with open(RUNS_FILE, "rb") as f:
    successful_runs = pickle.load(f)

In [14]:
for i, (config, trajectory) in enumerate(successful_runs):
    print(f"\nRendering successful episode {i + 1}")
    env = gym.make("custom-intersection-v0", render_mode='human', config=config)
    obs, _ = env.reset()
    for obs, action in trajectory:
        env.step(action)
        env.render()
        time.sleep(0.05)
    env.close()


Rendering successful episode 1

Rendering successful episode 2

Rendering successful episode 3

Rendering successful episode 4

Rendering successful episode 5

Rendering successful episode 6

Rendering successful episode 7

Rendering successful episode 8

Rendering successful episode 9

Rendering successful episode 10

Rendering successful episode 11

Rendering successful episode 12

Rendering successful episode 13

Rendering successful episode 14

Rendering successful episode 15

Rendering successful episode 16

Rendering successful episode 17

Rendering successful episode 18

Rendering successful episode 19

Rendering successful episode 20


KeyboardInterrupt: 